# Exercise - VaR

In [1]:
import pandas as pd
import numpy as np

# Data 
##### (We will be looking at AAPL, META, NVDA and TSLA)

In [8]:
# Load Data
tiks = ['AAPL', 'META', 'NVDA', 'TSLA']
file_path = "spx_returns_weekly.xlsx"
df = pd.read_excel(file_path, sheet_name=2, usecols=tiks) 


# Diversification

### 1.1 Calculate Volatility, Empirical Var(0.05), Empirical CVaR(0.05)

In [22]:
volatility = df.std(ddof=1)
var05 = df.quantile(0.05)
cvar05 = df.apply(lambda x: x[x <= x.quantile(0.05)].mean())

summary = pd.DataFrame({'Volatility': volatility, 'VaR(0.05)': var05, 'CVaR(0.05)': cvar05})
summary

,Volatility,VaR(0.05),CVaR(0.05)
AAPL,0.038362,-0.056366,-0.083125
META,0.048722,-0.070012,-0.103196
NVDA,0.064246,-0.086853,-0.116455
TSLA,0.081323,-0.117397,-0.147814


#### 1.2 Equally weighted portfolio

In [20]:
ew_portfolio_returns = df.mean(axis=1) # Equally weighted portfolio
ew_portfolio_returns_vol  = ew_portfolio_returns.std(ddof=1) ## degree of freedom = 1 for sample std not population in that case it would be ddof=0
ew_portfolio_returns_var  = ew_portfolio_returns.quantile(0.05) 
ew_portfolio_returns_cvar = ew_portfolio_returns[ew_portfolio_returns <= ew_portfolio_returns_var].mean()

portfolio_summary = pd.DataFrame({
    'Volatility': [ew_portfolio_returns_vol],
    'VaR(0.05)': [ew_portfolio_returns_var],
    'CVaR(0.05)': [ew_portfolio_returns_cvar]
}, index=['Equally Weighted Portfolio']).round(4)

portfolio_summary

,Volatility,VaR(0.05),CVaR(0.05)
Equally Weighted Portfolio,0.0438,-0.0619,-0.085


In [21]:
combined = pd.concat([summary, portfolio_summary])
combined


,Volatility,VaR(0.05),CVaR(0.05)
AAPL,0.038362,-0.056366,-0.083125
META,0.048722,-0.070012,-0.103196
NVDA,0.064246,-0.086853,-0.116455
TSLA,0.081323,-0.117397,-0.147814
Equally Weighted Portfolio,0.043800,-0.061900,-0.085000


#### 1.3 Re-calculate 1.2, but this time drop your most volatile asset, and replace the portion it was getting with 0. (You could imagine we’re replacing the most volatile asset with a negligibly small risk-free rate.)





In [19]:
most_volatile = summary['Volatility'].idxmax()
print("Most volatile stock:", most_volatile)

Most volatile stock: TSLA


In [26]:
df_adj = df.copy()
df_adj[most_volatile] = 0
ew_portfolio_returns_adj = df_adj.mean(axis=1)
ew_portfolio_returns_adj_vol  = ew_portfolio_returns_adj.std(ddof=1)
ew_portfolio_returns_adj_var  = ew_portfolio_returns_adj.quantile(0.05)
ew_portfolio_returns_adj_cvar = ew_portfolio_returns_adj[ew_portfolio_returns_adj  <= ew_portfolio_returns_adj_var].mean()

adjusted_port_summary = pd.DataFrame({
    'Volatility': [ew_portfolio_returns_adj_vol],
    'VaR(0.05)': [ew_portfolio_returns_adj_var],
    'CVaR(0.05)': [ew_portfolio_returns_adj_cvar]
}, index=['EWP (One Asset Replaced by Risk-Free)']).round(4)

combined_results = pd.concat([summary, portfolio_summary, adjusted_port_summary])
combined_results

,Volatility,VaR(0.05),CVaR(0.05)
AAPL,0.038362,-0.056366,-0.083125
META,0.048722,-0.070012,-0.103196
NVDA,0.064246,-0.086853,-0.116455
TSLA,0.081323,-0.117397,-0.147814
Equally Weighted Portfolio,0.043800,-0.061900,-0.085000
EWP (One Asset Replaced by Risk-Free),0.030300,-0.042500,-0.060500


##### In comparing the answer here to 1.2, how much risk is your most volatile asset adding to the portfolio? Is this in line with the amount of risk we measured in the stand-alone risk-assessment of 1.1?

TSLA contributes about 30–31% of the portfolio’s total risk, despite only a 25% weight. This aligns with its high stand-alone volatility (8.13%), confirming that the most volatile stock drives a large part of portfolio risk.

When removed and replaced by a risk-free asset, portfolio volatility, VaR, and CVaR all drop substantially — showing that the high-volatility asset was the key driver of risk exposure.

# 2. Dynamic Measures

### 2.1 Conditional Statistics

#### Rolling Volatility

In [ ]:
dynamic_metrics_df = df.copy()
m = 26  # rolling window size
rolling_vol = dynamic_metrics_df.rolling(window=m, min_periods=m).std(ddof=1)  # σ_t using data through t
rolling_vol_pred = rolling_vol.shift(1)
rolling_vol_pred.dropna(inplace=True) #drop first 26 rows...
rolling_vol_pred

,AAPL,META,NVDA,TSLA
26,0.028078,0.029184,0.047337,0.048927
27,0.028469,0.029057,0.047782,0.049967
28,0.027327,0.031460,0.047875,0.048403
29,0.026254,0.031113,0.047745,0.048958
30,0.026017,0.031376,0.045734,0.048990
...,...,...,...,...
537,0.046115,0.051540,0.076226,0.100697
538,0.047947,0.054819,0.078598,0.097651
539,0.047564,0.057596,0.078556,0.096224
540,0.047741,0.057173,0.076363,0.077772


In [33]:
z05 = -1.65
phi_z05 = (1 / np.sqrt(2 * np.pi)) * np.exp(-0.5 * z05**2)  # ≈ 0.1031

normal_VaR = z05 * rolling_vol_pred
normal_CVaR = - (phi_z05 / 0.05) * rolling_vol_pred

# Step 4: End-of-sample results (latest volatility-based risk forecast)
end_stats = pd.DataFrame({
    'Forecasted Volatility (σ_t)': rolling_vol_pred.iloc[-1],
    'Normal VaR(0.05)': normal_VaR.iloc[-1],
    'Normal CVaR(0.05)': normal_CVaR.iloc[-1]
}).round(4)

print(end_stats)

      Forecasted Volatility (σ_t)  Normal VaR(0.05)  Normal CVaR(0.05)
AAPL                       0.0496           -0.0819            -0.1015
META                       0.0578           -0.0953            -0.1181
NVDA                       0.0827           -0.1365            -0.1692
TSLA                       0.0849           -0.1400            -0.1736


The conditional (forecasted) estimates reflect current risk levels — they respond quickly to spikes in volatility.

The unconditional estimates reflect average risk over the full period — they are smoother and less responsive.

Therefore, it’s completely expected that your rolling-window (m=26) VaR and CVaR are more negative than the long-term averages if recent volatility has increased.

In [36]:
from scipy.stats import norm
import matplotlib.pyplot as plt

QUANTILE = 0.05
WINDOW   = 26
z05 = -1.65  # already defined above

# 1) Vol forecasts (σ_{t|t-1})
sigma_roll_pred = rolling_vol_pred                           # from your previous cell (already shifted)
sigma_exp_pred  = df.expanding(WINDOW).std(ddof=1).shift(1)  # expanding vol using data through t-1

# Ensure VaR DataFrames are aligned 1:1 with df (same index & columns)
VaR_roll = (z05 * sigma_roll_pred).reindex_like(df)
VaR_exp  = (z05 * sigma_exp_pred).reindex_like(df)

# Hit test (no double shift)
hits_roll = (df < VaR_roll) & VaR_roll.notna()
hits_exp  = (df < VaR_exp)  & VaR_exp.notna()

hit_pct_roll = (hits_roll.sum() / hits_roll.count() * 100).round(2)
hit_pct_exp  = (hits_exp.sum()  / hits_exp.count()  * 100).round(2)

hit_results = pd.DataFrame({
    'Hit % (Expanding vol)': hit_pct_exp,
    'Hit % (Rolling vol, m=26)': hit_pct_roll
})
print(hit_results)


      Hit % (Expanding vol)  Hit % (Rolling vol, m=26)
AAPL                   4.24                       4.98
META                   4.61                       4.24
NVDA                   2.77                       4.24
TSLA                   6.27                       4.98
